# Calculo de principales indicadores usando python

A continuación se presentan los coódigos para el cálculo de los principales indicadores de la ENIGH 2020. Estan escritos para que el usuario los ejecute sin necesidad de cambio.

Para realizar este ejercicio solo seran necesatios los módulos de **pandas** y **numpy**

## Set-up

In [1]:
# Importamos librerias que vamos a utilizar
import pandas as pd
import matplotlib.pyplot as plt
import os

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Creamos directorios
data = '/home/jovyan/work/02_indicadores_desigualdad/00_enigh/00_data/'
outputs = '/home/jovyan/work/02_indicadores_desigualdad/00_enigh/03_outputs/'

### Ejecutamos script con funciones necesarias

In [3]:
%run 'functions.py' 

## Lectura de tablas de datos

Para leer las tablas que necesitamos para el analisis, se creo una funcíon, **enigh**, que nos ayuda a traernos direcatmente de la pagina de la enigh la tabla del año que necesitemos, sin necesidad de guardar el arhivo en nuestro espacio de trabajo.

En este ejercicio, tomaremos la tabla *concentrado hogar* del año *2020* y lo guardamos en un dataframe

In [4]:
df = enigh('concentradohogar','2020')

___

## 2. Cálculo de indicadores de ingreso
### 2.1. Ingreso corriente total promedio trimestral por hogar en deciiles de hogares y su coeficiente de GINI

A continuación, se presenta el código que calcula el ingreso corriente total promedio por hogar en deciles de hogares y su coeficiente de GINI.
El coeficiente de Gini es una medida de concentración del ingreso: toma valores entre cero y uno. Cuando el valor se acerca a uno, indica que hay mayor concentración del ingreso; en cambio cuando el valor del Gini se acerca a cero la concentración del ingreso es menor.

<blockquote><div style="background-color:#DCDCDC; color:#000000; font-style: normal; font-family: Georgia;">
Folioviv (Folio de la vivienda):

Consta de 10 dígitos: dos dígitos con la clave de la entidad federativa, uno con el ámbito (urbano, código diferente a 6; rural, código 6), cuatro dígitos del número consecutivo de la upm (unidad primaria de muestreo), un dígito de la decena de levantamiento y dos dígitos con un número consecutivo para la vivienda seleccionada.
</div></blockquote>


<blockquote><div style="background-color:#DCDCDC; color:#000000; font-style: normal; font-family: Georgia;">
Foliohog (Folio del hogar):

Dígito para identificar a los hogares en la vivienda: se asigna 1 al hogar principal (el que inicialmente
atendió la entrevista) o cuando en la vivienda solo habita un hogar; cuando se detecten varios hogares
(hogares adicionales) se les asignan dígitos del 2 al 5.
</div></blockquote>

In [5]:
# Seleccionamos variables de interes

concentrado = df[['folioviv', 'foliohog', 'ing_cor', 'ingtrab', 'trabajo',
                  'negocio', 'otros_trab', 'rentas', 'utilidad', 'arrenda',
                 'transfer', 'jubilacion', 'becas', 'donativos', 'remesas',
                 'bene_gob', 'transf_hog', 'trans_inst', 'estim_alqu',
                 'otros_ing', 'factor', 'upm', 'est_dis']]

###### Calculamos algunos números rapidos

In [6]:
print('Total de Hogares: ', df.factor.sum())

print('Tamaño de los deciles: ',  df.factor.sum()/10 )

print('Promedio de integrantes por familia: {:.3} '.format(df.tot_integ.mean()))

Total de Hogares:  35749659
Tamaño de los deciles:  3574965.9
Promedio de integrantes por familia: 3.55 


Para crear los deciles de ingreso, utilizamos la función *weighted_qcut* definida al inicio del notebook para obtener los deciles ponderados por el factor de expansión

In [7]:
concentrado['decil']=weighted_qcut(concentrado['ing_cor'], concentrado['factor'], 10, labels=False)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Una vez que creamos el dataframe con las variables que vamos a utilizar, tenemos que restar a gasto monerario corriente (gasto_mon) las variables de los conceptos excentos de iva.

Y para finalizar, vamos a multiplicar las variables por el factor de expansión para tener valores nacionales.

In [8]:
# Se crea una variable de entidad federativa y de total de integrantes

concentrado['integrantes']= df['tot_integ']*concentrado['factor']
concentrado['entidad']=df.loc[:,('folioviv')].apply(str).str[1:3]

###### Total de hogares

In [33]:
(concentrado['ing_cor']*concentrado['factor'].sum()).groupby(by='decil')
# /concentrado.factor.sum()

KeyError: 'decil'

In [23]:
concentrado.groupby(by='decil').agg({'ing_cor': ['sum', 'mean', 'min','max']})

ing_cor                                   
                   sum       mean       min           max
decil                                                    
0        95,098,660.47   9,911.27      0.00     13,955.40
1       161,067,415.31  16,888.69 13,955.50     19,665.67
2       205,662,374.23  22,260.24 19,666.25     24,885.24
3       248,423,474.86  27,565.85 24,885.24     30,325.61
4       302,578,396.92  33,364.03 30,326.07     36,623.02
5       350,460,356.82  40,112.21 36,623.84     44,016.79
6       431,648,187.13  48,702.27 44,017.17     53,858.56
7       530,095,365.90  60,616.97 53,861.20     68,532.98
8       671,463,060.63  80,280.14 68,533.46     96,103.06
9     1,261,415,144.96 160,792.24 96,106.98 10,702,107.40

In [31]:
deciles_final.to_excel(outputs + 'data/deciles_desagregado.xlsx')